# Example: Solving Ordinary Differential Equations

In this notebook we will use Python to solve differential equations numerically.

In [ ]:
// Import the required modules
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FSharp.Stats"
#r "nuget: FSOde"

// ... and open the required modules
open FsODE
open Plotly.NET
open FSharp.Stats

Installed Packages FSharp.Stats, 0.4.7 FSOde, 0.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
// First set the model context that remembers the solver method and it´s optiony 
let modelContext = //OdeContext()
    OdeSolverMethod.RK546M //RK547M()
    |> OdeContext

modelContext.SetStepSize(0.01)

## Sampling out of master equations

For some stories, we know the probability distribution. For example, if the story is that we are doing n
independent coin flips, each with probability p of landing heads, the number h

of heads is Binomially distributed. We could prove that this is the case, or look it up in a book. But sometimes, it is too difficult (or impossible) to prove what the distribution is, so we can numerically compute its properties by sampling out of the distribution. Sampling involved using a random number generator to simulate the story that generates the distribution. So, if you know the story and you have a computer handy, you can sample to be able to get a plot of your distribution, though you will not get the analytical form.

Let’s demonstrate this with the Binomial distribution. We will take n=25
and p=0.25 and compute P(h∣n,p), the probability of getting h heads in n flips, each with probability p of landing heads. We will draw 10, 30, 100, and 300 samples and plot them versus the expected Binomial distribution.

In [ ]:
// Simulate n_samples sets of n coin flips with prob. p of heads.
let simulateCoinflips n p size =
    let rnd = new System.Random()
    [|0 .. size - 1|]
    |> Array.map (fun _ ->
        Array.init n (fun x -> rnd.NextDouble())
        |> Array.countBy (fun x -> x < p)
        |> Array.tryFind (fun (isHead,count) -> isHead)
        |> fun res ->
            match res with
            | Some (isHead,count) -> count
            | None -> 0
    )

let size = [|30;100;1000;10000|]
let n = 25
let p = 0.25

let binomial = Distributions.Discrete.binomial 0.25 25

let binomialPoints =
    [|0 .. 25|]
    |> Array.map (fun x -> x,binomial.PDF x)

size
|> Array.map (fun s ->
    [
        simulateCoinflips 25 0.25 s
        |> Array.countBy id
        |> Array.map (fun (heads,count) -> heads, float count/ float s)
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromString "Orange")
        binomialPoints
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromString "Blue", Opacity = 0.5)
    ]
    |> Chart.combine
    |> Chart.withLegend false
    |> Chart.withXAxisStyle(TitleText = $"h<br>{s} samples")
    |> Chart.withYAxisStyle(TitleText = "P(h)")
)
|> Chart.Grid(2,2)
|> Chart.withSize (800,800)

<!-- Plotly chart will be drawn inside this DIV -->

As we can see, if we sample out of the probability distribution, we can approximately calculate the actual distribution. If we sample enough, the approximation is very good.

Sampling is such a powerful strategy that highly efficient algorithms with convenient APIs have been developed to sample out of named probability distributions. For example, we could have used np.random.binom() as a drop-in (and much more efficient) replacement for the simulate_coinflips() function above.

We will use the same strategy for solving master equations. We will find a way to sample out of the distribution that is governed by the master equation. This technique was pioneered by Dan Gillespie in the last 70s. For that reason, these sampling techniques are often called Gillespie simulations. The algorithm is sometimes referred to as a stochastic simulation algorithm, or SSA.

Here, we will explore how this algorithm works by looking at simple production of a protein.
## The dynamical equations

For simple protein production, we have the following reactions.

DNA→mRNA→protein

### Macroscale equations

As we’ve seen before, the deterministic dynamics, which describe mean concentrations over a large population of cells, are described by the ODEs

dmdtdpdt=βm−γmm,=βpm−γpp.

The same equations should hold if m
and p represent the mean numbers of cells; we would just have to appropriately rescale the constants. Assuming the m and p are now numbers (so we are not free to pick their units), we can nondimensionalize using γm

to nondimensionalize time. This leads to redefinition of parameters and variables

βm/γm→βm,βp/γm→βp,γmt→t.

The dimensionless equations are

dmdtdpdt=βm−m,=βpm−γp,

with γ=γp/γm

.
### The Master equation

We can write a master equation for these dynamics. In this case, each state is defined by an mRNA copy number m
and a protein copy number p. So, we will write a master equation for P(m,p,t)

.

dP(m,p,t)dt=βmP(m−1,p,t)+(m+1)P(m+1,p,t)−βmP(m,p,t)−mP(m,p,t)+βpmP(m,p−1,t)+γ(p+1)P(m,p+1,t)−βpmP(m,p,t)−γpP(m,p,t).

We implicitly define P(m,p,t)=0
if m<0 or p<0

. This is the master equation we will sample from using the stochastic simulation algorithm (SSA) or Gillespie algorithm.
## The Gillespie algorithm

The transition probabilities are also called propensities in the context of stochastic simulation. The propensity for a given transition, say indexed i
, is denoted as ai. The equivalence to notation we introduced for master equations is that if transition i results in the change of state from n′ to n, then ai=W(n∣n′)

.

To cast this problem for a Gillespie simulation, we can write each change of state (moving either the copy number of mRNA or protein up or down by 1 in this case) and their respective propensities.

reaction, rim→m+1,m→m−1,p→p+1,p→p−1,propensity, aiβmmβpmγp.

We will not carefully prove that the Gillespie algorithm samples from the probability distribution governed by the master equation, but will state the principles behind it. The basic idea is that events (such as those outlined above) are rare, discrete, separate events. I.e., each event is an arrival of a Poisson process. The Gillespie algorithm starts with some state, (m0,p0)
. Then a state change, any state change, will happen in some time Δt that has a certain probability distribution (which we will show is Exponential momentarily). The probability that the state change that happens is reaction j is proportional to aj. That is to say, state changes with high propensities are more likely to occur. Thus, choosing which of the n state changes happens in Δt is a matter of drawing an integer j in [1,n] where the probability of drawing j

is

aj∑iai.

Now, how do we determine how long the state change took? The probability density function describing that a given state change i
takes place in time t

is

P(t∣ai)=aie−ait,

since the time it takes for arrival of a Poisson process is Exponentially distributed. The probability that it has not arrived in time Δt
is the probability that the arrival time is greater than Δt

, given by the complementary cumulative distribution function for the Exponential distribution.

P(t>Δt∣ai)=∫∞ΔtdtP(t∣ai)=e−aiΔt.

Now, say we have n
processes that arrive in time t1,t2,…. The probability that none of them arrive before Δt

is

P(t1>Δt,t2>Δt,…)=P(t1>Δt)P(t2>Δt)⋯=∏ie−aiΔt=exp[−Δt∑iai].

This is the same as the probability of a single Poisson process with a=∑iai
not arriving before Δt. So, the probability that it does arrive in Δt is Exponentially distributed with mean (∑iai)−1

.

So, we know how to choose a state change and we also know how long it takes. The Gillespie algorithm then proceeds as follows.

    Choose an initial condition, e.g., m=p=0

.

Calculate the propensity for each of the enumerated state changes. The propensities may be functions of m
and p, so they need to be recalculated for every m and p

we encounter.

Choose how much time the reaction will take by drawing out of an exponential distribution with a mean equal to (∑iai)−1

. This means that a change arises from a Poisson process.

Choose what state change will happen by drawing a sample out of the discrete distribution where Pi=ai/(∑iai)

. In other words, the probability that a state change will be chosen is proportional to its propensity.

Increment time by the time step you chose in step 3.

Update the states according to the state change you choose in step 4.

If t

    is less than your pre-determined stopping time, go to step 2. Else stop.

Gillespie proved that this algorithm samples the probability distribution described by the master equation in his seminal papers in 1976 and 1977. (We recommend reading the latter.) You can also read a concise discussion of how the algorithm samples the master equation in section 4.2 of Del Vecchio and Murray.
## Coding up the Gillespie simulation

To code up the Gillespie simulation, we first make an an array that gives the changes in the counts of m
and p for each of the four reactions. This is a way of encoding the updates in the particle counts that we get from choosing the respective state changes.

In [ ]:
// Column 0 is change in m, column 1 is change in p
let simpleUpdate =
    [|
        [|1, 0|];  // Make mRNA transcript
        [|-1, 0|]; // Degrade mRNA
        [|0, 1|];  // Make protein
        [|0, -1|]  // Degrade protein
    |]

Next, we make a function that updates the array of propensities for each of the four reactions. We update the propensities (which are passed into the function as an argument) instead of instantiating them and returning them to save on memory allocation while running the code. It has the added benefit that it forces you to keep track of the indices corresponding to the update matrix. This helps prevent bugs. It will naturally be a function of the current population of molecules. It may in general also be a function of time, so we explicitly allow for time dependence (even though we will not use it in this simple example) as well.

In [ ]:
//Updates an array of propensities given a set of parameters and an array of populations.
let simplePropensities (propensities: int []) population t beta_m beta_p gamma =
    let m,p = population
    propensities.[0] <- beta_m     // Make mRNA transcript
    propensities.[1] <- m          // Degrade mRNA
    propensities.[2] <- beta_p * m // Make protein
    propensities.[3] <- gamma * p  // Degrade protein

### Making a draw

Finally, we write a general function that draws a choice of reaction and the time interval for that reaction. This is the heart of the Gillespie algorithm, so we will take some time to discuss speed. First, to get the time interval, we sample a random number from an exponential distribution with mean (∑iai)−1

. This is easily done using the np.random.exponential() function.

Next, we have to select which reaction will take place. This amounts to drawing a sample over the discrete distribution where Pi=ai(∑iai)−1
, or the probability of each reaction is proportional to its propensity. This can be done using scipy.stats.rv_discrete, which allows specification of an arbitrary discrete distribution. We will write a function to do this.

In [ ]:
let sampleDiscrete (probs: 'a []) =
    let q =  (new System.Random()).NextDouble()
    let mutable i = 0
    let mutable pSum = 0.
    while pSum < q do
        pSum <- pSum + probs.[i]
        i <- i + 1
    i - 1

Now we can write a function to do our draws.

In [ ]:
// Make dummy probs
let probs = [|0.1;0.3;0.4;0.05;0.15|]
sampleDiscrete probs

2

In [ ]:
let gillespieDraw propensityFunc propensities population t beta_m beta_p gamma =
    (**
    Draws a reaction and the time it took to do that reaction.

    Parameters
    ----------
    propensity_func : function
        Function with call signature propensity_func(population, t, *args)
        used for computing propensities. This function must return
        an array of propensities.
    propensities : ndarray
        Propensities for each reaction as a 1D Numpy array.
    population : ndarray
        Current population of particles
    t : float
        Value of the current time.
    args : tuple, default ()
        Arguments to be passed to `propensity_func`.

    Returns
    -------
    rxn : int
        Index of reaction that occured.
    time : float
        Time it took for the reaction to occur.
    *)
    // compute propensities
    propensityFunc propensities population t beta_m beta_p gamma
    // sum of propensities
    let  propsSum = propensities |> Array.sum
    let time = (FSharp.Stats.Distributions.Continuous.exponential propsSum).Sample()
    // compute discrete probabilities of each reaction
    let rxnProbs = propensities |> Array.map (fun p -> p / propsSum)
    // draw reaction from this distribution
    let rxn = sampleDiscrete rxnProbs
    rxn, time

## SSA time stepping

Now we are ready to write our main SSA loop. We will only keep the counts at pre-specified time points. This saves on RAM, and we really only care about the values at given time points anyhow.

Note that this function is generic. All we need to specify our system is the following.

    A function to compute the propensities

    How the updates for a given reaction are made

    Initial population

Additionally, we specify necessary parameters, an initial condition, and the time points at which we want to store our samples. So, providing the propensity function and update are analogous to providing the time derivatives when using scipy.integrate.odeint().

In [ ]:
let gillespieSSA propensityFunc (update: 'a[]) population0 (timePoints: float[]) beta_m beta_p gamma =
    (**
    Uses the Gillespie stochastic simulation algorithm to sample
    from probability distribution of particle counts over time.

    Parameters
    ----------
    propensity_func : function
        Function of the form f(params, t, population) that takes the current
        population of particle counts and return an array of propensities
        for each reaction.
    update : ndarray, shape (num_reactions, num_chemical_species)
        Entry i, j gives the change in particle counts of species j
        for chemical reaction i.
    population_0 : array_like, shape (num_chemical_species)
        Array of initial populations of all chemical species.
    time_points : array_like, shape (num_time_points,)
        Array of points in time for which to sample the probability
        distribution.
    args : tuple, default ()
        The set of parameters to be passed to propensity_func.

    Returns
    -------
    sample : ndarray, shape (num_time_points, num_chemical_species)
        Entry i, j is the count of chemical species j at time
        time_points[i].
    *)
    // initialize output
    let mutable popOut = Array.zeroCreate timePoints.Length

    // initialize and perform simulation
    let mutable iTime = 1
    let mutable i = 0
    let mutable t = timePoints.[0]
    let mutable population = population0
    let mutable propensities = Array.zeroCreate update.Length
    let mutable populationPrevious = Array.zeroCreate 1
    popOut <- population
    while i < timePoints.Length do
        while t < timePoints.[iTime] do
            // draw the event and time step
            let event, dt = gillespieDraw propensityFunc propensities population t beta_m beta_p gamma
            // update the population
            populationPrevious <- population
            population <- populationPrevious |> Array.map2 (+) update.[event]
            // increment time
            t <- t + dt
        // update the index
        i <- timePoints |> Array.findIndex (fun x -> x > t)
        // update the population
        popOut.[iTime] <- populationPrevious
        // increment index
        iTime <- i
    popOut





Stopped due to error


Error: input.fsx (46,64)-(46,78) typecheck warning This construct causes code to be less generic than indicated by the type annotations. The type variable 'a has been constrained to be type 'int[]'.
input.fsx (52,27)-(52,45) typecheck error Type mismatch. Expecting a
    'int'    
but given a
    'int[]'    
The types 'int' and 'int[]' cannot be unified.